Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

# **Clean the Dataset, look for NaN values, change them to 0 and drop columns with all NaN values**

In [5]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [6]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [7]:
df = df.drop(['EASE-MENT', 'APARTMENT_NUMBER', 'LAND_SQUARE_FEET', 'TAX_CLASS_AT_TIME_OF_SALE', 
              'ZIP_CODE'], axis=1)
df.isnull().sum()

BOROUGH                            0
NEIGHBORHOOD                       0
BUILDING_CLASS_CATEGORY            0
TAX_CLASS_AT_PRESENT               1
BLOCK                              0
LOT                                0
BUILDING_CLASS_AT_PRESENT          1
ADDRESS                            0
RESIDENTIAL_UNITS                  1
COMMERCIAL_UNITS                   1
TOTAL_UNITS                        1
GROSS_SQUARE_FEET                  1
YEAR_BUILT                        35
BUILDING_CLASS_AT_TIME_OF_SALE     0
SALE_PRICE                         0
SALE_DATE                          0
dtype: int64

In [8]:
df.fillna(0, inplace=True)
df.isnull().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [0]:
df['BOROUGH'] = df['BOROUGH'].astype(int)


In [0]:
df['YEAR_BUILT'] = df['YEAR_BUILT'].fillna(0).astype(int)

In [14]:
df.dropna()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,R4,"447 WEST 18TH STREET, PH12A",1.0,0.0,1.0,1979.0,2007,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,O5,144 WEST 37TH STREET,0.0,6.0,6.0,15435.0,1920,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,O5,40 WEST 38TH STREET,0.0,7.0,7.0,11332.0,1930,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,R4,"1 SHERIDAN SQUARE, 8C",1.0,0.0,1.0,500.0,0,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,R1,"20 EAST 65TH STREET, B",1.0,0.0,1.0,6406.0,0,R1,0,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,4,OTHER,01 ONE FAMILY DWELLINGS,1,10965,276,A5,111-17 FRANCIS LEWIS BLVD,1.0,0.0,1.0,1224.0,1945,A5,510000,04/30/2019
23036,4,OTHER,09 COOPS - WALKUP APARTMENTS,2,169,29,C6,"45-14 43RD STREET, 3C",0.0,0.0,0.0,0.0,1929,C6,355000,04/30/2019
23037,4,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,131,4,D4,"50-05 43RD AVENUE, 3M",0.0,0.0,0.0,0.0,1932,D4,375000,04/30/2019
23038,4,OTHER,02 TWO FAMILY DWELLINGS,1,8932,18,S2,91-10 JAMAICA AVE,2.0,1.0,3.0,2200.0,1931,S2,1100000,04/30/2019


In [15]:
df.dtypes

BOROUGH                             int64
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                          int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

# **Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.**

In [18]:
new_subset = (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)

df = df[new_subset]

df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,1.0,0.0,1.0,1325.0,1930,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,1.0,0.0,1.0,2001.0,1940,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,1.0,0.0,1.0,2043.0,1925,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,1.0,0.0,1.0,2680.0,1899,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,1.0,0.0,1.0,1872.0,1940,A5,620000,01/02/2019


# **Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.**

In [19]:
#Change sale date from Obj to DateTime

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
df.dtypes

BOROUGH                                    int64
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                                 int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE                         datetime64[ns]
dtype: object

In [20]:
# train will be dates Jan-Mar 2019 then Test will be just April 2019

train = df[df['SALE_DATE'].dt.month <= 3]
test = df[df['SALE_DATE'].dt.month == 4]

train.shape, test.shape

((2517, 16), (647, 16))

# **Do one-hot encoding of categorical features.**

In [21]:
#Show only Numeric columns 

train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BOROUGH,2517.0,3.911005,0.896483,1.0,3.0,4.0,5.0,5.0
BLOCK,2517.0,6750.859754,3977.126707,21.0,3818.0,6021.0,9879.0,16350.0
LOT,2517.0,75.674215,157.232723,1.0,21.0,42.0,70.0,2720.0
RESIDENTIAL_UNITS,2517.0,0.987684,0.110315,0.0,1.0,1.0,1.0,1.0
COMMERCIAL_UNITS,2517.0,0.016289,0.129712,0.0,0.0,0.0,0.0,2.0
TOTAL_UNITS,2517.0,1.003973,0.171453,0.0,1.0,1.0,1.0,3.0
GROSS_SQUARE_FEET,2517.0,1473.500596,599.132016,0.0,1144.0,1367.0,1683.0,7875.0
YEAR_BUILT,2517.0,1944.729837,27.054972,1890.0,1925.0,1940.0,1960.0,2018.0
SALE_PRICE,2517.0,621011.272149,295079.398887,100000.0,440000.0,560000.0,750000.0,2000000.0


In [22]:
#Show only non-numeric columns

train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2517,1,01 ONE FAMILY DWELLINGS,2517,NaT,NaT
TAX_CLASS_AT_PRESENT,2517,2,1,2486,NaT,NaT
NEIGHBORHOOD,2517,7,OTHER,2368,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2517,11,A1,921,NaT,NaT
BUILDING_CLASS_AT_PRESENT,2517,13,A1,921,NaT,NaT
SALE_DATE,2517,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
ADDRESS,2517,2507,118-20 202ND STREET,2,NaT,NaT


In [23]:
train.groupby('NEIGHBORHOOD')['SALE_PRICE'].mean()

NEIGHBORHOOD
ASTORIA               1.001955e+06
BEDFORD STUYVESANT    5.636420e+05
BOROUGH PARK          1.008917e+06
EAST NEW YORK         4.171025e+05
FLUSHING-NORTH        8.590834e+05
FOREST HILLS          1.210753e+06
OTHER                 6.073127e+05
Name: SALE_PRICE, dtype: float64

In [24]:
train.groupby('NEIGHBORHOOD')['SALE_PRICE'].describe()

,count,mean,std,min,25%,50%,75%,max
NEIGHBORHOOD,,,,,,,,
ASTORIA,11.0,1.001955e+06,365110.014554,232000.0,820250.00,1090000.0,1180000.0,1600000.0
BEDFORD STUYVESANT,9.0,5.636420e+05,384325.334432,100000.0,250000.00,600000.0,780000.0,1128508.0
BOROUGH PARK,12.0,1.008917e+06,340243.284172,600000.0,898250.00,913750.0,988500.0,1850000.0
EAST NEW YORK,22.0,4.171025e+05,148082.271677,210000.0,264616.25,445000.0,498500.0,800000.0
FLUSHING-NORTH,78.0,8.590834e+05,246849.236539,100000.0,769250.00,845000.0,930000.0,1695000.0
FOREST HILLS,17.0,1.210753e+06,392811.007543,640000.0,875000.00,1122000.0,1490000.0,1876000.0
OTHER,2368.0,6.073127e+05,285207.260523,100000.0,435000.00,550000.0,720000.0,2000000.0


In [0]:
import category_encoders as ce

target = 'SALE_PRICE'
high_card = ['ADDRESS', 'SALE_DATE', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE']
features = train.columns.drop([target] + high_card)

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [28]:
X_train

,BOROUGH,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT
44,3,1,0,0,0,0,0,0,1,1,0,5495,801,1.0,0.0,1.0,1325.0,1930
61,4,1,0,0,0,0,0,0,1,1,0,7918,72,1.0,0.0,1.0,2001.0,1940
78,2,1,0,0,0,0,0,0,1,1,0,4210,19,1.0,0.0,1.0,2043.0,1925
108,3,1,0,0,0,0,0,0,1,1,0,5212,69,1.0,0.0,1.0,2680.0,1899
111,3,1,0,0,0,0,0,0,1,1,0,7930,121,1.0,0.0,1.0,1872.0,1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,5,1,0,0,0,0,0,0,1,1,0,4081,44,1.0,0.0,1.0,921.0,1950
18130,5,1,0,0,0,0,0,0,1,1,0,2373,201,1.0,0.0,1.0,2128.0,1980
18132,5,1,0,0,0,0,0,0,1,1,0,1132,42,1.0,0.0,1.0,1807.0,2018
18134,5,1,0,0,0,0,0,0,1,1,0,3395,37,1.0,0.0,1.0,621.0,1930


In [29]:
X_test

,BOROUGH,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT
18235,2,1,0,0,0,0,0,0,1,1,0,5913,878,1.0,0.0,1.0,2272.0,1930
18239,2,1,0,0,0,0,0,0,1,1,0,5488,48,1.0,0.0,1.0,720.0,1935
18244,3,1,0,0,0,0,0,0,1,1,0,5936,31,1.0,0.0,1.0,2210.0,1925
18280,3,1,0,0,0,0,0,0,1,1,0,7813,24,1.0,0.0,1.0,1520.0,1915
18285,3,1,0,0,0,0,0,0,1,1,0,8831,160,1.0,0.0,1.0,840.0,1925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,1,0,0,0,0,0,0,1,1,0,13215,3,1.0,0.0,1.0,1478.0,1925
23031,4,1,0,0,0,0,0,0,1,1,0,11612,73,1.0,0.0,1.0,1280.0,1930
23032,4,1,0,0,0,0,0,0,1,1,0,11808,50,1.0,0.0,1.0,1333.0,1945
23033,4,1,0,0,0,0,0,0,1,1,0,12295,23,1.0,0.0,1.0,1020.0,1935


In [30]:
features = X_train.columns
n = len(features)
n

18

# **Do feature selection with SelectKBest.**

In [33]:
from sklearn.feature_selection import SelectKBest, f_regression

select = SelectKBest(score_func=f_regression, k=10)

X_train_select = select.fit_transform(X_train, y_train)
X_test_select = select.transform(X_test)

X_train_select.shape, X_test_select.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


((2517, 10), (647, 10))

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

select_mask = select.get_support()
all_names = X_train.columns
selected_names = all_names[select_mask]
unselected_names = all_names[~select_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\nFeatures not selected:')
for name in unselected_names:
    print(name)

Features selected:
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
TAX_CLASS_AT_PRESENT_1
BLOCK
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET

Features not selected:
BOROUGH
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_ASTORIA
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
TAX_CLASS_AT_PRESENT_1D
LOT
YEAR_BUILT


In [40]:

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $185,788 

2 features
Test Mean Absolute Error: $180,408 

3 features
Test Mean Absolute Error: $179,901 

4 features
Test Mean Absolute Error: $177,856 

5 features
Test Mean Absolute Error: $177,866 

6 features
Test Mean Absolute Error: $176,970 

7 features
Test Mean Absolute Error: $178,469 

8 features
Test Mean Absolute Error: $174,499 

9 features
Test Mean Absolute Error: $174,499 

10 features
Test Mean Absolute Error: $174,375 

11 features
Test Mean Absolute Error: $174,322 

12 features
Test Mean Absolute Error: $174,898 

13 features
Test Mean Absolute Error: $175,132 

14 features
Test Mean Absolute Error: $175,142 

15 features
Test Mean Absolute Error: $174,686 

16 features
Test Mean Absolute Error: $173,325 

17 features
Test Mean Absolute Error: $173,325 

18 features
Test Mean Absolute Error: $173,325 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

# ** Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)**

In [0]:
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, Ridge

